In [13]:
from util import load_user_data
import pandas as pd
import sys
import numpy as np

In [14]:
valid_users, tp_data, _ = load_user_data(filename="../data/xailabdata_all.csv")
valid_users_2, tp_data_2, _ = load_user_data(filename="../data/xailabdata_llm_agent.csv")

valid_users = valid_users | valid_users_2
tp_data.update(tp_data_2)

all_conditions = ["control", "dashboard", "chatxai", "chatxaiboost", "chatxaiAuto"]
condition_confidence_dynamics_before = {}
condition_confidence_dynamics_after = {}
number_tasks = 10
for condition in all_conditions:
    condition_confidence_dynamics_after[condition] = []
    condition_confidence_dynamics_before[condition] = []
    for i in range(number_tasks):
        condition_confidence_dynamics_after[condition].append([])
        condition_confidence_dynamics_before[condition].append([])
condition_confidence_before = {}
condition_confidence_after  = {}
for condition in all_conditions:
    condition_confidence_before[condition] = []
    condition_confidence_after[condition]  = []
variable_dict = {
    "condition": [],
    "initial_confidence": [],
    "final_confidence": []
}
user_confidence_vec = {}
for user in valid_users:
    user_confidence_vec[user] = {}
    tp_condition = tp_data[user]["condition"]
    user_task_order = tp_data[user]["task_order"]
    user_confidence = tp_data[user]["confidence"]
    variable_dict["condition"].append(tp_condition)
    # print(user_confidence)
    try:
        assert len(user_task_order) == 10
    except:
        print(len(user_task_order))
    # print(data_long_format[tp_condition])
    # print(tp_condition, len(data_long_format[tp_condition]["confidence"]))
    confidence_before_advice = []
    confidence_after_advice = []
    for index, task_id in enumerate(user_task_order):
        tp_confidence = int(user_confidence[(task_id, "advice")])
        # assert isinstance(tp_confidence, int)
        condition_confidence_dynamics_after[tp_condition][index].append(tp_confidence)
        confidence_after_advice.append(tp_confidence)

        tp_confidence = int(user_confidence[(task_id, "base")])
        condition_confidence_dynamics_before[tp_condition][index].append(tp_confidence)
        confidence_before_advice.append(tp_confidence)
    user_confidence_vec[user]["initial"] = confidence_before_advice
    user_confidence_vec[user]["final"] = confidence_after_advice
    condition_confidence_after[tp_condition].append(np.mean(confidence_after_advice))
    condition_confidence_before[tp_condition].append(np.mean(confidence_before_advice))
    variable_dict["initial_confidence"].append(np.mean(confidence_before_advice))
    variable_dict["final_confidence"].append(np.mean(confidence_after_advice))
# for condition in all_conditions:
#     print(len(condition_confidence_before[condition]), len(condition_confidence_after[condition]))
print(len(variable_dict["condition"]))

245 valid participants
{'control': 61, 'dashboard': 61, 'chatxai': 62, 'chatxaiboost': 61, 'chatxaiAuto': 0}
91 participants blindly rely on AI advice
{'control': 8, 'dashboard': 27, 'chatxai': 32, 'chatxaiboost': 24, 'chatxaiAuto': 0}
61 valid participants
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 61}
25 participants blindly rely on AI advice
{'control': 0, 'dashboard': 0, 'chatxai': 0, 'chatxaiboost': 0, 'chatxaiAuto': 25}
306


In [17]:
from scipy.stats import kruskal, mannwhitneyu
def post_hoc_comparison(data_list_1, data_list_2, name1, name2):
	print("Use pots-hoc analysis")
	threshold = 0.05 / 4
	flag = False
	statistic, pvalue = mannwhitneyu(data_list_1, data_list_2, alternative='greater')
	if pvalue < threshold:
		print("Alternative {} > {},".format(name1, name2), "pvalue %.4f"%pvalue, "statistic %.4f"%statistic)
		flag = True
	statistic, pvalue = mannwhitneyu(data_list_1, data_list_2, alternative='less')
	if pvalue < threshold:
		print("Alternative {} < {},".format(name1, name2), "pvalue %.4f"%pvalue, "statistic %.4f"%statistic)
		flag = True
	if not flag:
		print("No significant difference with post-hoc analysis")

In [26]:
from scipy.stats import kstest, ks_2samp, ttest_rel, wasserstein_distance
from scipy.spatial.distance import jensenshannon 
# import jensenshannon, wasserstein_distance
import sys
condition_dict = {}
for condition in all_conditions:
    condition_dict[condition] = []
for user in valid_users:
    tp_condition = tp_data[user]["condition"]
    confidence_before_advice = [item for item in user_confidence_vec[user]["initial"]]
    confidence_after_advice  = [item for item in user_confidence_vec[user]["final"]]
    # res = ks_2samp(confidence_before_advice, confidence_after_advice)
    # stat, pvalue = res
    # print(stat)
    # t_stat, p_value = ttest_rel(confidence_before_advice, confidence_after_advice)
    # print(f"Paired t-test: t-statistic = {t_stat}, p-value = {p_value}")
    # sys.exit(-1)
    emd = wasserstein_distance(confidence_before_advice, confidence_after_advice)
    # emd = jensenshannon(confidence_before_advice, confidence_after_advice)
    # print(f"Earth Mover's Distance (Wasserstein Distance): {emd}")
    condition_dict[tp_condition].append(emd)

In [27]:
kwargs = [condition_dict[condition] for condition in all_conditions]
statistic, pvalue = kruskal(*kwargs)
print("kruskal test result: H:{:.2f}, p:{:.3f}".format(statistic, pvalue))
tp_str = "Final" + "&" + "{:.2f} & {:.3f}& ".format(statistic, pvalue)
for condition in all_conditions:
    data_list_1 = condition_dict[condition]
    print("{}, Mean: M({}):{:.2f}, SD({}):{:.2f}".format(len(data_list_1), condition, np.mean(data_list_1), condition, np.std(data_list_1)))
    tp_str += "${:.2f} \\pm {:.2f}$ &".format(np.mean(data_list_1), np.std(data_list_1))
print(tp_str)
if pvalue < 0.05 / 4:
    length = len(all_conditions)
    for i in range(length - 1):
        for j in range(i+1, length):
            group_1 = all_conditions[i]
            group_2 = all_conditions[j]
            data_list_1 = condition_dict[group_1]
            data_list_2 = condition_dict[group_2]
            post_hoc_comparison(data_list_1, data_list_2, group_1, group_2)
print("-" * 17)

kruskal test result: H:5.59, p:0.232
61, Mean: M(control):0.35, SD(control):0.32
61, Mean: M(dashboard):0.42, SD(dashboard):0.28
62, Mean: M(chatxai):0.39, SD(chatxai):0.31
61, Mean: M(chatxaiboost):0.44, SD(chatxaiboost):0.31
61, Mean: M(chatxaiAuto):0.48, SD(chatxaiAuto):0.40
Final&5.59 & 0.232& $0.35 \pm 0.32$ &$0.42 \pm 0.28$ &$0.39 \pm 0.31$ &$0.44 \pm 0.31$ &$0.48 \pm 0.40$ &
-----------------


In [5]:
df = pd.DataFrame(variable_dict)

from pingouin import ancova, anova
from statsmodels.stats.multicomp import pairwise_tukeyhsd
for dimension in ["initial_confidence", "final_confidence"]:
    print(dimension)
    aov = anova(dv=dimension, between='condition', data=df, effsize='n2')
    print(aov.round(3))
    if aov.to_dict()['p-unc'][0] <= 0.05 / 4:
        tukey = pairwise_tukeyhsd(endog=df[dimension], groups=df['condition'], alpha=0.0125)
        print(tukey)



initial_confidence
      Source  ddof1  ddof2      F  p-unc     n2
0  condition      4    302  0.483  0.748  0.006
final_confidence
      Source  ddof1  ddof2      F  p-unc     n2
0  condition      4    302  0.651  0.627  0.009


/Users/ghe/opt/anaconda3/envs/agreement_phi/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [6]:
kwargs = [condition_confidence_before[condition] for condition in all_conditions]
statistic, pvalue = kruskal(*kwargs)
print("kruskal test result: H:{:.2f}, p:{:.3f}".format(statistic, pvalue))
tp_str = "Initial" + "&" + "{:.2f} & {:.3f}& ".format(statistic, pvalue)
for condition in all_conditions:
    data_list_1 = condition_confidence_before[condition]
    print("{}, Mean: M({}):{:.2f}, SD({}):{:.2f}".format(len(data_list_1), condition, np.mean(data_list_1), condition, np.std(data_list_1)))
    tp_str += "${:.2f} \\pm {:.2f}$ &".format(np.mean(data_list_1), np.std(data_list_1))
print(tp_str)
if pvalue < 0.05 / 4:
    length = len(all_conditions)
    for i in range(length - 1):
        for j in range(i+1, length):
            group_1 = all_conditions[i]
            group_2 = all_conditions[j]
            data_list_1 = condition_confidence_before[group_1]
            data_list_2 = condition_confidence_before[group_2]
            post_hoc_comparison(data_list_1, data_list_2, group_1, group_2)
print("-" * 17)

kruskal test result: H:4.50, p:0.342
61, Mean: M(control):3.86, SD(control):0.71
61, Mean: M(dashboard):3.85, SD(dashboard):0.54
62, Mean: M(chatxai):4.00, SD(chatxai):0.68
61, Mean: M(chatxaiboost):3.91, SD(chatxaiboost):0.66
62, Mean: M(chatxaiAuto):3.93, SD(chatxaiAuto):0.75
Initial&4.50 & 0.342& $3.86 \pm 0.71$ &$3.85 \pm 0.54$ &$4.00 \pm 0.68$ &$3.91 \pm 0.66$ &$3.93 \pm 0.75$ &
-----------------


In [7]:
kwargs = [condition_confidence_after[condition] for condition in all_conditions]
statistic, pvalue = kruskal(*kwargs)
print("kruskal test result: H:{:.2f}, p:{:.3f}".format(statistic, pvalue))
tp_str = "Final" + "&" + "{:.2f} & {:.3f}& ".format(statistic, pvalue)
for condition in all_conditions:
    data_list_1 = condition_confidence_after[condition]
    print("{}, Mean: M({}):{:.2f}, SD({}):{:.2f}".format(len(data_list_1), condition, np.mean(data_list_1), condition, np.std(data_list_1)))
    tp_str += "${:.2f} \\pm {:.2f}$ &".format(np.mean(data_list_1), np.std(data_list_1))
print(tp_str)
if pvalue < 0.05 / 4:
    length = len(all_conditions)
    for i in range(length - 1):
        for j in range(i+1, length):
            group_1 = all_conditions[i]
            group_2 = all_conditions[j]
            data_list_1 = condition_confidence_after[group_1]
            data_list_2 = condition_confidence_after[group_2]
            post_hoc_comparison(data_list_1, data_list_2, group_1, group_2)
print("-" * 17)

kruskal test result: H:4.57, p:0.334
61, Mean: M(control):4.07, SD(control):0.63
61, Mean: M(dashboard):4.04, SD(dashboard):0.52
62, Mean: M(chatxai):4.13, SD(chatxai):0.64
61, Mean: M(chatxaiboost):4.12, SD(chatxaiboost):0.58
62, Mean: M(chatxaiAuto):4.20, SD(chatxaiAuto):0.63
Final&4.57 & 0.334& $4.07 \pm 0.63$ &$4.04 \pm 0.52$ &$4.13 \pm 0.64$ &$4.12 \pm 0.58$ &$4.20 \pm 0.63$ &
-----------------
